In [1]:
import os
from os.path import expanduser
import socket
import sys
import glob

# BELOW IS SYSTEM/PATH CONFIGURATION #
my_home = expanduser("~")
running_system = socket.gethostname()
# Configure machine's names here #
systemHLRN = 'blogin1'
systemLOCAL = 'Laurents-MacBook-Pro.local'
my_home = expanduser("~")
# Define local pass #
if running_system == systemLOCAL:
    print "computer = ", running_system
    wd_path = os.path.join(my_home,"Documents","projects", "pyfesom")
elif running_system[0:1] == "b" or running_system == systemHLRN:
    print "computer = ", running_system
    wd_path = os.path.join(my_home,"pyfesom")
else:
    print 'please configure your local machine : type socket.gethostname()'
print "working directory set to", wd_path
os.chdir(wd_path)
sys.path.append("codes/modules") # add custom Vibe 's modules
#sys.path.append("pyfesom") # add standard 's modules
#########################################

computer =  blogin1
working directory set to /home/hbkoziel/pyfesom


In [2]:
import matplotlib.pyplot as plt
from mpl_toolkits.basemap import Basemap
from pylab import *
import pyfesom as pf
import numpy as np
import datetime as dt
from PiecewiseNorm import PiecewiseNorm
from netCDF4 import Dataset
#import cmocean as cmaps
from matplotlib.colors import ListedColormap
import colormaps as cmaps
from matplotlib.path import Path
import matplotlib.patches as patches

No joblib
no cmocean


In [3]:
# PLOT CONFIGURATION #

# print definition
dpicnt = 150
# load colormaps #
cmap = ListedColormap(cmaps.viridis.colors)
# choose simulation
runid='Arc12'
# choose depth
get3d = False
# variable ID
mld_id = 'mixlay'
din_id= 'tr18'
chl1_id, chl2_id = 'tr06', 'tr15'
ice_id= 'area'
pp1_id, pp2_id = 'diags_bio1_mean','diags_bio2_mean'
temp_id = 'temp'
sal_id = 'salt'
# choose date
first_year = 2003
last_year  = 2015
years      = np.arange(first_year,last_year+1,1)
########################

# AUTOMATIC DEFINITION OF PATHS
resultpath = '/scratch/usr/hbkvsk12/hlrn3_work2/results/'+runid+'/Oldfiles/'
savepath    = wd_path+'/figures/'+runid+'/'
meshpath    = wd_path+'/data/mesh/meshArc4.5/'


# Create figure directory if it does not exist
if os.path.exists(savepath) == False:
    try:
        os.mkdir(savepath)
        print ("Creation of the directory %s successfull" % savepath)
    except OSError:
        print ("Creation of the directory %s failed" % savepath)
else:
    print ("directory %s already existing" % savepath)

directory /home/hbkoziel/pyfesom/figures/Arc12/ already existing


In [4]:
mesh = pf.load_mesh(meshpath, usepickle=True,get3d=get3d)

/home/hbkoziel/pyfesom/data/mesh/meshArc4.5/pickle_mesh
2
The usepickle == True)
The pickle file for python 2 exists.
The mesh will be loaded from /home/hbkoziel/pyfesom/data/mesh/meshArc4.5/pickle_mesh


In [ ]:
ind_box = np.where((mesh.x2>=40) & (mesh.x2<=60) & (mesh.y2>=75) & (mesh.y2<=80))

for year in years:
    ncfile = resultpath+runid+'.'+str(year)+'.ice.mean.nc'
    fice      = Dataset(ncfile, 'r')
    ncfile = resultpath+runid+'.'+str(year)+'.oce.mean.nc'
    foce      = Dataset(ncfile, 'r')
    ncfile = resultpath+runid+'.'+str(year)+'.oce.diag.nc'
    foced      = Dataset(ncfile, 'r')
    ncfile = resultpath+runid+'.'+str(year)+'.bio.mean.nc'
    fbio      = Dataset(ncfile, 'r')

    ice    = fice.variables[ice_id][:,0:mesh.n2d]
    ice_mean = np.squeeze(ice[:,ind_box]).mean(axis=1)
    pp    = fbio.variables[pp1_id][:,0:mesh.n2d]+fbio.variables[pp2_id][:,0:mesh.n2d]
    pp = pp*12.04
    pp_mean = np.squeeze(pp[:,ind_box]).mean(axis=1)
    #chl    = foce.variables[chl1_id][:,0:mesh.n2d]+foce.variables[chl2_id][:,0:mesh.n2d]
    #chl_mean = foce.variables[chl1_id][:,0:mesh.n2d].mean(axis=1)+foce.variables[chl2_id][:,0:mesh.n2d].mean(axis=1)
    mld    = foced.variables[mld_id][:,0:mesh.n2d]
    mld_mean = np.squeeze(mld[:,ind_box]).mean(axis=1)
    din    = foce.variables[din_id][:,0:mesh.n2d]
    din_mean = np.squeeze(din[:,ind_box]).mean(axis=1)
    
    for doy in range(0,din.shape[0]):
        
        # PLOTING PART #
        rows,cols = 3,2
        factor = 4
        figsize = (rows*factor, cols*factor)
        fig = plt.figure(constrained_layout=True, figsize=figsize)
        fig.tight_layout()
        gs = GridSpec(rows, cols, figure=fig)

        ax1 = fig.add_subplot(gs[0, 0])
        contours = np.array([0, 20, 40, 80, 100, 150, 350, 500, 750, 1000, 1500, 2000])
        #contours = [0, 1200, 10]
        #contours = np.arange(contours[0], contours[1]+contours[2], contours[2])
        cmap='Blues'
        data2=mld[doy,:]
        elem2=mesh.elem[mesh.no_cyclic_elem,:]
        d=data2[elem2].mean(axis=1)
        k = [i for (i, val) in enumerate(d) if not np.isnan(val)]
        elem2=elem2[k,:]
        print 'ftriplot, number of dummy points:', len(d)-len(k)		
        map = Basemap(width=12000000,height=9000000,
                    rsphere=(6378137.00,6356752.3142),\
                    resolution='l',area_thresh=1000.,projection='lcc',\
                    llcrnrlon=10.,llcrnrlat=60.,urcrnrlon=110.,urcrnrlat=85.,\
                    lat_0=75,lon_0=55., ax = ax1)
        x, y = map(mesh.x2, mesh.y2)
        #map.drawcoastlines()
        plabels=[False,False,False,False]
        mlabels=[False,False,False,False]    
        map.drawparallels(np.arange(60.,81.,20.), labels=plabels)
        map.drawmeridians(np.arange(-60.,181.,20.),labels=mlabels) #[0,1,0,0]
        map.drawmapboundary(fill_color='0.9')
        map.fillcontinents(color='.5',lake_color='.7')
        eps=(contours.max()-contours.min())/100.
        data2[data2<=contours.min()]=contours.min()+eps
        data2[data2>=contours.max()]=contours.max()-eps
        # plot 1
        plt.tricontour(x, y, elem2, ice[doy,:],levels=([0.9]),colors='white',linewidths=2)
        plt.tricontour(x, y, elem2, ice[doy,:],levels=([0.9]),colors='black',linewidths=1)
        plt.tricontour(x, y, elem2, ice[doy,:],levels=([0.1]),colors='white',linewidths=2)
        plt.tricontour(x, y, elem2, ice[doy,:],levels=([0.1]),colors='black',linewidths=1,linestyles='--')
        im1=plt.tricontourf(x, y, elem2, data2, levels=contours, cmap=cmap, norm=PiecewiseNorm(contours))
        cb1 = fig.colorbar(im1, ax=ax1)
        cb1.set_ticks(contours)

        #Drawing the zoom rectangles:

        lbx1, lby1 = map(40, 75)
        ltx1, lty1 = map(40, 80)
        rtx1, rty1 = map(60, 80)
        rbx1, rby1 = map(60, 75)

        verts1 = [
            (lbx1, lby1), # left, bottom
            (ltx1, lty1), # left, top
            (rtx1, rty1), # right, top
            (rbx1, rby1), # right, bottom
            (lbx1, lby1), # ignored
            ]

        codes2 = [Path.MOVETO,
                 Path.LINETO,
                 Path.LINETO,
                 Path.LINETO,
                 Path.CLOSEPOLY,
                 ]

        path = Path(verts1, codes2)
        patch = patches.PathPatch(path, facecolor='r', alpha=0.4)
        ax1.add_patch(patch)

        ax2 = fig.add_subplot(gs[1, 0])
        cmap='magma'
        contours = [0, 12, 1]
        contours = np.arange(contours[0], contours[1]+contours[2], contours[2])
        #contours = np.array([0, 50, 100, 150, 200, 250, 500, 750, 1000, 1500, 2000])
        data2=din[doy,:]
        elem2=mesh.elem[mesh.no_cyclic_elem,:]
        d=data2[elem2].mean(axis=1)
        k = [i for (i, val) in enumerate(d) if not np.isnan(val)]
        elem2=elem2[k,:]
        print 'ftriplot, number of dummy points:', len(d)-len(k)		
        map = Basemap(width=12000000,height=9000000,
                    rsphere=(6378137.00,6356752.3142),\
                    resolution='l',area_thresh=1000.,projection='lcc',\
                    llcrnrlon=10.,llcrnrlat=60.,urcrnrlon=110.,urcrnrlat=85.,\
                    lat_0=75,lon_0=55., ax = ax2)
        x, y = map(mesh.x2, mesh.y2)
        #map.drawcoastlines()
        plabels=[False,False,False,False]
        mlabels=[False,False,False,False]    
        map.drawparallels(np.arange(60.,81.,20.), labels=plabels)
        map.drawmeridians(np.arange(-60.,181.,20.),labels=mlabels) #[0,1,0,0]
        map.drawmapboundary(fill_color='0.9')
        map.fillcontinents(color='.5',lake_color='.7')
        eps=(contours.max()-contours.min())/100.
        data2[data2<=contours.min()]=contours.min()+eps
        data2[data2>=contours.max()]=contours.max()-eps
        # plot 2
        plt.tricontour(x, y, elem2, ice[doy,:],levels=([0.9]),colors='white',linewidths=2)
        plt.tricontour(x, y, elem2, ice[doy,:],levels=([0.9]),colors='black',linewidths=1)
        plt.tricontour(x, y, elem2, ice[doy,:],levels=([0.1]),colors='white',linewidths=2)
        plt.tricontour(x, y, elem2, ice[doy,:],levels=([0.1]),colors='black',linewidths=1,linestyles='--')
        im2=plt.tricontourf(x, y, elem2, data2, levels=contours, cmap=cmap, norm=PiecewiseNorm(contours))
        cb2 = fig.colorbar(im2, ax=ax2)
        cb2.set_ticks(contours)


        ax3 = fig.add_subplot(gs[2, 0])
        cmap='Greens'
        contours = np.array([0, 50, 100, 150, 200, 250, 500, 750, 1000, 1500, 2000])
        #contours = [0, 500, 1]
        #contours = np.arange(contours[0], contours[1]+contours[2], contours[2])
        data2=pp[doy,:]
        elem2=mesh.elem[mesh.no_cyclic_elem,:]
        d=data2[elem2].mean(axis=1)
        k = [i for (i, val) in enumerate(d) if not np.isnan(val)]
        elem2=elem2[k,:]
        print 'ftriplot, number of dummy points:', len(d)-len(k)		
        map = Basemap(width=12000000,height=9000000,
                    rsphere=(6378137.00,6356752.3142),\
                    resolution='l',area_thresh=1000.,projection='lcc',\
                    llcrnrlon=10.,llcrnrlat=60.,urcrnrlon=110.,urcrnrlat=85.,\
                    lat_0=75,lon_0=55., ax = ax3)
        x, y = map(mesh.x2, mesh.y2)
        #map.drawcoastlines()
        plabels=[False,False,False,False]
        mlabels=[False,False,False,False]    
        map.drawparallels(np.arange(60.,81.,20.), labels=plabels)
        map.drawmeridians(np.arange(-60.,181.,20.),labels=mlabels) #[0,1,0,0]
        map.drawmapboundary(fill_color='0.9')
        map.fillcontinents(color='.5',lake_color='.7')
        eps=(contours.max()-contours.min())/100.
        data2[data2<=contours.min()]=contours.min()+eps
        data2[data2>=contours.max()]=contours.max()-eps
        # plot 3
        plt.tricontour(x, y, elem2, ice[doy,:],levels=([0.9]),colors='white',linewidths=2)
        plt.tricontour(x, y, elem2, ice[doy,:],levels=([0.9]),colors='black',linewidths=1)
        plt.tricontour(x, y, elem2, ice[doy,:],levels=([0.1]),colors='white',linewidths=2)
        plt.tricontour(x, y, elem2, ice[doy,:],levels=([0.1]),colors='black',linewidths=1,linestyles='--')
        im3=plt.tricontourf(x, y, elem2, data2, levels=contours, cmap=cmap, norm=PiecewiseNorm(contours))
        cb3 = fig.colorbar(im3, ax=ax3)  
        cb3.set_ticks(contours)

        ax4 = fig.add_subplot(gs[0, 1])
        if ((year >= 2000) & (year<2015)):
            ax4.plot(np.arange(1,351,2),mld_mean, color = 'tab:blue')
            ax4.plot(np.arange(1,351,2)[doy],mld_mean[doy], marker = '.', markersize = 10, color = 'tab:red')
            x = np.arange(0,350,2)
            plt.title(str(x[doy])+'-'+str(year))
        elif year == 2015:
            ax4.plot(np.arange(1,335,2),mld_mean, color = 'tab:blue')
            ax4.plot(np.arange(1,335,2)[doy],mld_mean[doy], marker = '.', markersize = 10, color = 'tab:red')
            x = np.arange(0,334,2)
            plt.title(str(x[doy])+'-'+str(year))
        elif year <2000:
            ax4.plot(np.arange(15,350,30),mld_mean, color = 'tab:blue')
            ax4.plot(np.arange(15,350,30)[doy],mld_mean[doy], marker = '.', markersize = 10, color = 'tab:red')
            x = np.arange(15,350,30)
            plt.title(str(x[doy])+'-'+str(year))
        ax4.set_ylabel('MLD (m)')

        ax5 = fig.add_subplot(gs[1, 1])
        if ((year >= 2000) & (year<2015)):
            ax5.plot(np.arange(1,351,2),din_mean, color = 'tab:purple')
            ax5.plot(np.arange(1,351,2)[doy],din_mean[doy], marker = '.', markersize = 10, color = 'tab:red')
        elif year == 2015:
            ax5.plot(np.arange(1,335,2),din_mean, color = 'tab:purple')
            ax5.plot(np.arange(1,335,2)[doy],din_mean[doy], marker = '.', markersize = 10, color = 'tab:red')
        elif year <2000:
            ax5.plot(np.arange(15,350,30),din_mean, color = 'tab:purple')
            ax5.plot(np.arange(15,350,30)[doy],din_mean[doy], marker = '.', markersize = 10, color = 'tab:red')
        ax5.set_ylabel('DIN ($\mu$M)')

        ax6 = fig.add_subplot(gs[2, 1])
        if ((year >= 2000) & (year<2015)):
            ax6.plot(np.arange(1,351,2),pp_mean, color = 'tab:green')
            ax6.plot(np.arange(1,351,2)[doy],pp_mean[doy], marker = '.', markersize = 10, color = 'tab:red')
        elif year == 2015:
            ax6.plot(np.arange(1,335,2),pp_mean, color = 'tab:green')
            ax6.plot(np.arange(1,335,2)[doy],pp_mean[doy], marker = '.', markersize = 10, color = 'tab:red')
        elif year <2000:
            ax6.plot(np.arange(15,350,30),pp_mean, color = 'tab:green')
            ax6.plot(np.arange(15,350,30)[doy],pp_mean[doy], marker = '.', markersize = 10, color = 'tab:red')

        ax6.set_ylabel('NPP (mgC m$^{-2}$ day$^{-1}$)')
        ax6.set_xlabel('day of year')
        
        if ((year >= 2000) & (year<2015)):
            x = np.arange(1,350,2)
            if ((x[doy] >= 10) & (x[doy] < 99)):
                plt.savefig(savepath+'GINseas/NEBS/NEBS_'+str(year)+'0'+str(x[doy])+'.png', dpi = dpicnt, bbox_inches='tight')
            elif x[doy] < 9:
                plt.savefig(savepath+'GINseas/NEBS/NEBS_'+str(year)+'00'+str(x[doy])+'.png', dpi = dpicnt, bbox_inches='tight')
            elif x[doy] >= 100:
                plt.savefig(savepath+'GINseas/NEBS/NEBS_'+str(year)+str(x[doy])+'.png', dpi = dpicnt, bbox_inches='tight')
        elif year == 2015:
            x = np.arange(1,335,2)
            if ((x[doy] >= 10) & (x[doy] < 99)):
                plt.savefig(savepath+'GINseas/NEBS/NEBS_'+str(year)+'0'+str(x[doy])+'.png', dpi = dpicnt, bbox_inches='tight')
            elif x[doy] < 9:
                plt.savefig(savepath+'GINseas/NEBS/NEBS_'+str(year)+'00'+str(x[doy])+'.png', dpi = dpicnt, bbox_inches='tight')
            elif x[doy] >= 100:
                plt.savefig(savepath+'GINseas/NEBS/NEBS_'+str(year)+str(x[doy])+'.png', dpi = dpicnt, bbox_inches='tight')
        elif year <2000:
            x = np.arange(15,350,30)
            if ((x[doy] >= 10) & (x[doy] < 99)):
                plt.savefig(savepath+'GINseas/NEBS/NEBS_'+str(year)+'0'+str(x[doy])+'.png', dpi = dpicnt, bbox_inches='tight')
            elif x[doy] < 9:
                plt.savefig(savepath+'GINseas/NEBS/NEBS_'+str(year)+'00'+str(x[doy])+'.png', dpi = dpicnt, bbox_inches='tight')
            elif x[doy] >= 100:
                plt.savefig(savepath+'GINseas/NEBS/NEBS_'+str(year)+str(x[doy])+'.png', dpi = dpicnt, bbox_inches='tight')
        
        plt.close()

/sw/tools/anaconda2/2019.10/skl/lib/python2.7/site-packages/matplotlib/figure.py:2117: UserWarning: This figure was using constrained_layout==True, but that is incompatible with subplots_adjust and or tight_layout: setting constrained_layout==False. 
  warnings.warn("This figure was using constrained_layout==True, "


ftriplot, number of dummy points: 0
ftriplot, number of dummy points: 0
ftriplot, number of dummy points: 0
ftriplot, number of dummy points: 0
ftriplot, number of dummy points: 0
ftriplot, number of dummy points: 0
ftriplot, number of dummy points: 0
ftriplot, number of dummy points: 0
ftriplot, number of dummy points: 0
ftriplot, number of dummy points: 0
ftriplot, number of dummy points: 0
ftriplot, number of dummy points: 0
ftriplot, number of dummy points: 0
ftriplot, number of dummy points: 0
ftriplot, number of dummy points: 0
ftriplot, number of dummy points: 0
ftriplot, number of dummy points: 0
ftriplot, number of dummy points: 0
ftriplot, number of dummy points: 0
ftriplot, number of dummy points: 0
ftriplot, number of dummy points: 0
ftriplot, number of dummy points: 0
ftriplot, number of dummy points: 0
ftriplot, number of dummy points: 0
ftriplot, number of dummy points: 0
ftriplot, number of dummy points: 0
ftriplot, number of dummy points: 0
ftriplot, number of dummy po

ftriplot, number of dummy points: 0
ftriplot, number of dummy points: 0
ftriplot, number of dummy points: 0
ftriplot, number of dummy points: 0
ftriplot, number of dummy points: 0
ftriplot, number of dummy points: 0
ftriplot, number of dummy points: 0
ftriplot, number of dummy points: 0
ftriplot, number of dummy points: 0
ftriplot, number of dummy points: 0
ftriplot, number of dummy points: 0
ftriplot, number of dummy points: 0
ftriplot, number of dummy points: 0
ftriplot, number of dummy points: 0
ftriplot, number of dummy points: 0
ftriplot, number of dummy points: 0
ftriplot, number of dummy points: 0
ftriplot, number of dummy points: 0
ftriplot, number of dummy points: 0
ftriplot, number of dummy points: 0
ftriplot, number of dummy points: 0
ftriplot, number of dummy points: 0
ftriplot, number of dummy points: 0
ftriplot, number of dummy points: 0
ftriplot, number of dummy points: 0
ftriplot, number of dummy points: 0
ftriplot, number of dummy points: 0
ftriplot, number of dummy po

ftriplot, number of dummy points: 0
ftriplot, number of dummy points: 0
ftriplot, number of dummy points: 0
ftriplot, number of dummy points: 0
ftriplot, number of dummy points: 0
ftriplot, number of dummy points: 0
ftriplot, number of dummy points: 0
ftriplot, number of dummy points: 0
ftriplot, number of dummy points: 0
ftriplot, number of dummy points: 0
ftriplot, number of dummy points: 0
ftriplot, number of dummy points: 0
ftriplot, number of dummy points: 0
ftriplot, number of dummy points: 0
ftriplot, number of dummy points: 0
ftriplot, number of dummy points: 0
ftriplot, number of dummy points: 0
ftriplot, number of dummy points: 0
ftriplot, number of dummy points: 0
ftriplot, number of dummy points: 0
ftriplot, number of dummy points: 0
ftriplot, number of dummy points: 0
ftriplot, number of dummy points: 0
ftriplot, number of dummy points: 0
ftriplot, number of dummy points: 0
ftriplot, number of dummy points: 0
ftriplot, number of dummy points: 0
ftriplot, number of dummy po

ftriplot, number of dummy points: 0
ftriplot, number of dummy points: 0
ftriplot, number of dummy points: 0
ftriplot, number of dummy points: 0
ftriplot, number of dummy points: 0
ftriplot, number of dummy points: 0
ftriplot, number of dummy points: 0
ftriplot, number of dummy points: 0
ftriplot, number of dummy points: 0
ftriplot, number of dummy points: 0
ftriplot, number of dummy points: 0
ftriplot, number of dummy points: 0
ftriplot, number of dummy points: 0
ftriplot, number of dummy points: 0
ftriplot, number of dummy points: 0
ftriplot, number of dummy points: 0
ftriplot, number of dummy points: 0
ftriplot, number of dummy points: 0
ftriplot, number of dummy points: 0
ftriplot, number of dummy points: 0
ftriplot, number of dummy points: 0
ftriplot, number of dummy points: 0
ftriplot, number of dummy points: 0
ftriplot, number of dummy points: 0
ftriplot, number of dummy points: 0
ftriplot, number of dummy points: 0
ftriplot, number of dummy points: 0
ftriplot, number of dummy po

ftriplot, number of dummy points: 0
ftriplot, number of dummy points: 0
ftriplot, number of dummy points: 0
ftriplot, number of dummy points: 0
ftriplot, number of dummy points: 0
ftriplot, number of dummy points: 0
ftriplot, number of dummy points: 0
ftriplot, number of dummy points: 0
ftriplot, number of dummy points: 0
ftriplot, number of dummy points: 0
ftriplot, number of dummy points: 0
ftriplot, number of dummy points: 0
ftriplot, number of dummy points: 0
ftriplot, number of dummy points: 0
ftriplot, number of dummy points: 0
ftriplot, number of dummy points: 0
ftriplot, number of dummy points: 0
ftriplot, number of dummy points: 0
ftriplot, number of dummy points: 0
ftriplot, number of dummy points: 0
ftriplot, number of dummy points: 0
ftriplot, number of dummy points: 0
ftriplot, number of dummy points: 0
ftriplot, number of dummy points: 0
ftriplot, number of dummy points: 0
ftriplot, number of dummy points: 0
ftriplot, number of dummy points: 0
ftriplot, number of dummy po

ftriplot, number of dummy points: 0
ftriplot, number of dummy points: 0
ftriplot, number of dummy points: 0
ftriplot, number of dummy points: 0
ftriplot, number of dummy points: 0
ftriplot, number of dummy points: 0
ftriplot, number of dummy points: 0
ftriplot, number of dummy points: 0
ftriplot, number of dummy points: 0
ftriplot, number of dummy points: 0
ftriplot, number of dummy points: 0
ftriplot, number of dummy points: 0
ftriplot, number of dummy points: 0
ftriplot, number of dummy points: 0
ftriplot, number of dummy points: 0
ftriplot, number of dummy points: 0
ftriplot, number of dummy points: 0
ftriplot, number of dummy points: 0
ftriplot, number of dummy points: 0
ftriplot, number of dummy points: 0
ftriplot, number of dummy points: 0
ftriplot, number of dummy points: 0
ftriplot, number of dummy points: 0
ftriplot, number of dummy points: 0
ftriplot, number of dummy points: 0
ftriplot, number of dummy points: 0
ftriplot, number of dummy points: 0
ftriplot, number of dummy po

ftriplot, number of dummy points: 0
ftriplot, number of dummy points: 0
ftriplot, number of dummy points: 0
ftriplot, number of dummy points: 0
ftriplot, number of dummy points: 0
ftriplot, number of dummy points: 0
ftriplot, number of dummy points: 0
ftriplot, number of dummy points: 0
ftriplot, number of dummy points: 0
ftriplot, number of dummy points: 0
ftriplot, number of dummy points: 0
ftriplot, number of dummy points: 0
ftriplot, number of dummy points: 0
ftriplot, number of dummy points: 0
ftriplot, number of dummy points: 0
ftriplot, number of dummy points: 0
ftriplot, number of dummy points: 0
ftriplot, number of dummy points: 0
ftriplot, number of dummy points: 0
ftriplot, number of dummy points: 0
ftriplot, number of dummy points: 0
ftriplot, number of dummy points: 0
ftriplot, number of dummy points: 0
ftriplot, number of dummy points: 0
ftriplot, number of dummy points: 0
ftriplot, number of dummy points: 0
ftriplot, number of dummy points: 0
ftriplot, number of dummy po

ftriplot, number of dummy points: 0
ftriplot, number of dummy points: 0
ftriplot, number of dummy points: 0
ftriplot, number of dummy points: 0
ftriplot, number of dummy points: 0
ftriplot, number of dummy points: 0
ftriplot, number of dummy points: 0
ftriplot, number of dummy points: 0
ftriplot, number of dummy points: 0
ftriplot, number of dummy points: 0
ftriplot, number of dummy points: 0
ftriplot, number of dummy points: 0
ftriplot, number of dummy points: 0
ftriplot, number of dummy points: 0
ftriplot, number of dummy points: 0
ftriplot, number of dummy points: 0
ftriplot, number of dummy points: 0
ftriplot, number of dummy points: 0
ftriplot, number of dummy points: 0
ftriplot, number of dummy points: 0
ftriplot, number of dummy points: 0
ftriplot, number of dummy points: 0
ftriplot, number of dummy points: 0
ftriplot, number of dummy points: 0
ftriplot, number of dummy points: 0
ftriplot, number of dummy points: 0
ftriplot, number of dummy points: 0
ftriplot, number of dummy po